In [1]:
import os
import re
import bs4
import time
import requests
import pprint
import pickle

def load(url):
    res = requests.get(url)
    res.raise_for_status()

    return res.text

def pickup_tag(html, find_tag):
    soup = bs4.BeautifulSoup(str(html), 'html.parser')
    paragraphs = soup.find_all(find_tag)

    return paragraphs

def parse(html):
    soup = bs4.BeautifulSoup(str(html), 'html.parser')
    # htmlタグの排除
    kashi_row = soup.getText()
    kashi_row = kashi_row.replace('\n', '')
    kashi_row = kashi_row.replace('　', '')

    # 英数字の排除
    kashi_row = re.sub(r'[a-zA-Z0-9]', '', kashi_row)
    # 記号の排除
    kashi_row = re.sub(r'[ 「」＜＞♪`‘’“”・…_！？!-/:-@[-`{-~]', '', kashi_row)
    # 注意書きの排除
    kashi = re.sub(r'注意：.+', '', kashi_row)

    return kashi

def main():
    # 歌詞を格納
    lyrics_list = []
    
    # アーティストページのアドレス
    url_list = [f'https://www.uta-net.com/artist/3315/0/1/',f'https://www.uta-net.com/artist/3315/0/2/',f'https://www.uta-net.com/artist/3315/0/3/'] # ←これは3ページ目を参照している
    
    # 曲ページの先頭アドレス
    base_url = f'https://www.uta-net.com'

    # ページごとにfor文で回して取得した歌詞をlyrics_listに格納する
    for url in url_list:
        # ページの取得
        html = load(url)

        # 曲ごとのurlを格納
        musics_url = []

        """ 曲のurlを取得 """
        # td要素の取り出し
        for td in pickup_tag(html, 'td'):
            # a要素の取り出し
            for a in pickup_tag(td, 'a'):
                # href属性にsongを含むか
                if 'song' in a.get('href'): 
                    # urlを配列に追加
                    musics_url.append(base_url + a.get('href'))
        # pprint.pprint(musics_url)

        """ 歌詞の取得 """
        for i, page in enumerate(musics_url):
            print('{}曲目:{}'.format(i + 1, page))
            html = load(page)
            for div in pickup_tag(html, 'div'):
                # id検索がうまく行えなかった為、一度strにキャスト
                div = str(div)
                # 歌詞が格納されているdiv要素か
                if r'itemprop="text"' in div:
                    # 不要なデータを取り除く
                    kashi = parse(div)
                    kashi = kashi[:-34] # 'この歌詞をマイ歌ネットに登録～～～'を削除

                    print(kashi, end = '\n\n')
                    # 歌詞リストへ歌詞の書き込み
                    lyrics_list.append(kashi)

                    # １秒待機
                    time.sleep(1)
                    break
    # pickleで保存
    f = open('lyrics_list.txt', 'wb')
    pickle.dump(lyrics_list, f)

if __name__ == '__main__':
    main()

1曲目:https://www.uta-net.com/song/328718/
手すりのない橋を全力で走る怖こわいのは足元の深い峡谷を見るせいだ透きとおった道を全力で走る硝子ガラスかも氷かも疑いが足をすくませるつんのめって出遅れて日は沈む雨は横なぐりだ倶ともに走りだそう倶ともに走り継つごう過ぎた日々の峡谷をのぞき込むヒマはもうない倶ともに走りだそう倶ともに走り継つごう生きる互いの気配がただひとつだけの灯火ともしび身代みがわりは要いらない背負わなくてもいい手を引いてこちらへと示してほしいわけでもない君は走っているぜったい走ってる確かめるすべもない遠い遠い距離の彼方かなたで独ひとりずつ独ひとりずつ僕たちは全力で共鳴する倶ともに走りだそう倶ともに走り継つごう風前の灯火ともしびだとしても消えるまできっちり点ともっていたい倶ともに走りだそう倶ともに走り継つごう生きる互いの気配がただひとつだけの灯火ともしびつんのめって出遅れて日は沈む雨は横なぐりだ倶ともに走りだそう倶ともに走り継つごう風前の灯火ともしびだとしても消えるまできっちり点ともっていたい倶ともに走りだそう倶ともに走り継つごう生きる互いの気配がただひとつだけの灯火ともしび生きる互いの気配がただひとつだけの灯火ともしび

2曲目:https://www.uta-net.com/song/54640/
愛から遙か遠く離れてしまう時がこの世にあると思いもせずいられた自分が懐かしい愛から遙か遠く離れて生きる時は時計を海に捨てに行こう永遠のリフレインに一番好きな服を着て一番好きな私でいよういつか或る日思いがけず船が出るかもしれないから愛から遙か遠く離れて生きる時は時計を海に捨てに行こう永遠のリフレインに愛から遙か遠く離れて生きる人は寄せては返す波と同じどこへも逃げぬ囚とらわれ人びと愛から遙か遠く離れて生きる時は時計を海に捨てに行こう永遠のリフレインに一番好きな人がいた一番好きな私がいたいつか全ての思い出が遙かな海へと変わるから愛から遙か遠く離れて生きる時は時計を海に捨てに行こう永遠のリフレインに愛から遙か遠く離れて生きる時は時計を海に捨てに行こう永遠のリフレインに

3曲目:https://www.uta-net.com/song/87369/
すべて愛が私たちに命ずることならためらいはしない怖れもなく後も見ず歩いてゆけるだろう

21曲目:https://www.uta-net.com/song/231/
ララバイひとりで眠れない夜はララバイあたしをたずねておいてララバイひとりで泣いてちゃみじめよララバイ今夜はどこからかけてるの春は菜の花秋には桔梗そしてあたしはいつも夜咲くアザミララバイひとりで泣いてちゃみじめよララバイ今夜はどこからかけてるのララバイなんにも考えちゃいけないララバイ心におおいをかけてララバイおやすみ涙をふいてララバイおやすみ何もかも忘れて春は菜の花秋には桔梗そしてあたしはいつも夜咲くアザミララバイおやすみ涙をふいてララバイおやすみ何もかも忘れて春は菜の花秋には桔梗そしてあたしはいつも夜咲くアザミララバイひとりで眠れない夜はララバイあたしをたずねておいでララバイひとりで泣いてちゃみじめよララバイ今夜はどこからかけてるのララバイララバイララバイラララララバイララバイララバイララララララバイララバイララバイラララ

22曲目:https://www.uta-net.com/song/236/
イヤリングを外して綺麗じゃなくなってもまだ私のことを見失ってしまわないでねフリルのシャツを脱いでやせっぽちになってもまだ私のことを見失ってしまわないでねカーラジオが嵐を告げている人は黙りこんでいる形のないものに誰が愛なんてつけたのだろう教えてよもしも明日私たちが何もかも失くしてただの心しか持たないやせた猫になってももしも明日あなたのため何の得もなくても言えるならその時愛を聞かせて抱きしめれば人はなお遠くなるみたい許し合えば人はなおわからなくなるみたいだガラスならあなたの手の中で壊れたいナイフならあなたを傷つけながら折れてしまいたい何もかも愛を追い越してくどしゃ降りの車線の人生凍えながら人共が人分傷ついている教えてよもしも明日私たちが何もかも失くしてただの心しか持たないやせた猫になってももしも明日あなたのため何の得もなくても言えるならその時愛を聞かせて何もかも愛を追い越してくどしゃ降りの車線の人生凍えながら人共が人分傷ついている教えてよもしも明日私たちが何もかも失くしてただの心しか持たないやせた猫になってももしも明日あなたのため何の得もなくても言えるならその時愛を聞かせて

23曲目:https://www.uta-net.com/song/240/
なんにつけ一応は絶望的観測をするのが癖です

そうよだましたのは私心こわれたのは貴方どうせあなたも嘘つきな旅人と思ってたのよ裏切られた思い出にいつか覚えた氷芝居さみしがり屋の貴方にはそれが一番の仕打ちだった冷たい雨、雨、雨、雨、いまさら貴方がこんなにいとしい冷たい雨、雨、雨、雨、私をあの頃に連れて戻って生まれてはじめて会う人が貴方ならよかったけれど裏切られすぎて私は今さら素顔になれない裏切られた思い出にいつか覚えた氷芝居さみしがり屋の貴方にはそれが一番の仕打ちだった冷たい雨、雨、雨、雨、いまさら貴方がこんなにいとしい冷たい雨、雨、雨、雨、私をあの頃に連れて戻ってこわれた心をかかえて貴方は優しい女に出逢う幸せになってゆくならなんにも言えやしないけど裏切られた思い出にいつか覚えた氷芝居さみしがり屋の貴方にはそれが一番の仕打ちだった冷たい雨、雨、雨、雨、いまさら貴方がこんなにいとしい冷たい雨、雨、雨、雨、私をあの頃に連れて戻って

43曲目:https://www.uta-net.com/song/54152/
雨が空を捨てる日は忘れた昔が戸を叩く忘れられない優しさで車が着いたと夢を告げる空は風色ため息模様人待ち顔の店じまい雨が空を見限ってあたしの心にのり換える雨が空を捨てる日は直しあきらめる首飾りひとつふたつとつなげても必ず終わりが見あたらない空は風色ため息模様人待ち顔の店じまい雨が空を見限ってあたしの心に降りしきる空は風色ため息模様人待ち顔の店じまい雨が空を見限ってあたしの心に降りしきる

44曲目:https://www.uta-net.com/song/239945/
やまぬ雨のように考え続けようあなたのことだけを考え続けよう世の中のことなど振り向きもせず不安を予感して泣く赤ん坊たち不安を予感して恋する大人たち未来は嘘をつく予感を嗤わらう１人では歌は歌えない受けとめられて産まれるあてもなく夜の空へ鳥を放つかのようにあてもなく声に出す心を放つ１人では歌は歌えない受けとめられて産まれる響きあう波を探してアリア人は誰も自由になりたかった海辺で聴く歌は波の音ねと似ている砂漠で聴く歌は砂の音ねと似ている私の歌声は何にでもなる私は待ちわびるあなたの歌声を私は待ちわびる誤あやまたずあなたを騒がしい時代に埋もれぬように１人では歌は歌えない受けとめられて産まれる巡りあう人の総すべて愛せるほど強くない巡りあう人の総すべて敵なら虚

63曲目:https://www.uta-net.com/song/36811/
この一生だけでは辿り着けないとしても命のバトン掴つかんで願いを引き継いでゆけごらん夜空を星の線路がガラスの笛を吹いて通過信号を出す虫も獣も人も魚も透明なゴール目指す次の宇宙へと繋つなぐこの一生だけでは辿り着けないとしても命のバトン掴つかんで願いを引き継いでゆけこの一生だけでは辿り着けないとしても命のバトン掴つかんで願いを引き継いでゆけ僕の命を僕は見えないいつのまに走り始めいつまでを走るのだろう星も礫こいしも人も木の葉もひとつだけ運んでゆく次のスタートへ繋つなぐこの一生だけでは辿り着けないとしても命のバトン掴つかんで願いを引き継いでゆけこの一生だけでは辿り着けないとしても命のバトン掴つかんで願いを引き継いでゆけこの一生だけでは辿り着けないとしても命のバトン掴つかんで願いを引き継いでゆけ

64曲目:https://www.uta-net.com/song/55040/
初めて出会ったとき不思議な気がした鏡よりもっと似てる人をみつけた気がした離れず暮らすことが自然だった日々命ある限りずっと二人でいると思ってた妹じゃあるまいし別れたらそれまで笑顔も泣き顔も別れたらそれまで思いもよらぬうちに傷つけることをどれくらいあの人に浴びせ続けたのかしらもしも妹にならもっと早い日にあの人もつらくなる前に何か言えたはず妹じゃあるまいしケンカなんかしないでもその代わり二人にはサヨナラだけだった妹じゃあるまいし別れたらそれまで笑顔も泣き顔も別れたらそれまで妹じゃあるまいし別れたらそれまで笑顔も泣き顔も別れたらそれまで

65曲目:https://www.uta-net.com/song/172313/
次の次の北風が吹けば次の峰を越えてゆけるひとつひとつ北風を待って羽ばたきをやめないさみしい心先頭を飛んで弱い心中にかばって信じる心いちばん後から歌いながら飛ぶよほら次の雪風にあおられて小さな小さな鳥の列がなぎ払われる小さな小さな鳥の列が組み直される飛びたて飛びたて戻る場所はもうない飛びたて飛びたて夜の中へ強い鳥は雪が来る前に既すでに峰を越えて行った薄い羽根を持つ鳥たちは逆風を見上げるいつの風か約束はされないいちばん強い逆風だけが高く高く峰を越えるだろう羽ばたきはやまないほら次の雪風にあおられて小さな小さな鳥の列が

84曲目:https://www.uta-net.com/song/54580/
縁ある人万里の道を越えて引き合うもの縁なき人顔をあわせすべもなくすれ違うあなたを私は追い回す私はあなたの毒になる河よ教えて泣く前にこの縁はありやなしやあなたを私は追い回す私はあなたの毒になる河よ教えて泣く前にこの縁はありやなしやこの縁はありやなしや

85曲目:https://www.uta-net.com/song/54793/
このままでいいじゃないかあなたの煙草切り札を躱されて私の煙草唇で溶けあってそこからこぼれるのはため息ばかりの遠雷の夜答などを求めないそんな女はいないわあぁ今夜もほだされて舞い戻る腕の中せつなく乱されて遠ざかる雨の音後ろ手の置き手紙ちぎれてもう一夜なりゆきにまかせてこれ以上は望み過ぎなのね遠雷の夜いとしさがきわまって何かしでかしそうなのにあぁ今夜もほだされて舞い戻る腕の中せつなく乱されて遠ざかる雨の音ガレージの車には違う口紅がある見ないふりするのもこれまでよ言いかけたのにあぁ今夜もほだされて舞い戻る腕の中せつなく乱されて遠ざかる雨の音あぁ今夜もほだされて舞い戻る腕の中せつなく乱されて遠ざかる雨の音

86曲目:https://www.uta-net.com/song/54557/
追いかけてヨコハマあの人が逃げる残した捨てゼリフに誰か見覚えはありませんか追いかけてヨコハマあの人がいつもこの街をほめたことだけが裏切りの手がかりです旅の仕度をした人ばかりどうしてこんなに通るのでしょうヨコハマヨコハマこの船は街ごと運んで旅ですか追いかけてヨコハマあの人が逃げる残した捨てゼリフに誰か見覚えはありませんか追いかけてヨコハマ心当たりにはひとつ残らず寄ったけれど返事はなぐさめばかりこの街を最後にあの人のことで私にわかっていることは何ひとつなくなります旅の終わりはヨコハマあたり溜息ためいき街だと言いましたヨコハマヨコハマ似た街はどこかにそんなにありますか追いかけてヨコハマあの人が逃げる残した捨てゼリフに誰か見覚えはありませんか旅の仕度をした人ばかりどうしてこんなに通るのでしょうヨコハマヨコハマこの船は街ごと運んで旅ですか追いかけてヨコハマあの人が逃げる残した捨てゼリフに誰か見覚えはありませんか残した捨てゼリフに誰か見覚えはありませんか

87曲目:https://www.ut

...

179曲目:https://www.uta-net.com/song/157755/
瞬きひとつのあいだの一生僕たちはみんな一瞬の星瞬きもせずに息をすることさえ惜しむかのように求めあうああ君は何時ここから消えるのああ君は何時僕をおいて去ってしまうの僕をおいて何処へゆくの君を映す鏡の中君を誉める歌はなくても僕は誉める君の知らぬ君についていくつでもあのささやかな人生を良くは言わぬ人もあるだろうあのささやかな人生を無駄となじる人もあるだろうでも僕は誉める君の知らぬ君についていくつでも瞬きひとつのあいだの一生僕たちはみんな一瞬の星瞬きもせずに息をすることさえ惜しむかのように求めあうああ人は獣牙も毒も棘もなくただ痛むための涙だけを持って生まれた裸すぎる獣たちだ触れようとされるだけで痛む人は火傷してるから通り過ぎる街の中でそんな人を見かけないかあのささやかな人生はもしかしたら僕に似ているあのささやかな人生はもしかしたら君だったのか通り過ぎる街の中でそんな人を見かけないか瞬きひとつのあいだの一生僕たちはみんな一瞬の星瞬きもせずに息をすることさえ惜しむかのように求めあうああ人は獣牙も毒も棘もなくただ痛むための涙だけを持って生まれた裸すぎる獣たちだから僕は誉める君の知らぬ君についていくつでも瞬きひとつのあいだの一生僕たちはみんな一瞬の星瞬きもせずに息をすることさえ惜しむかのように求めあう

180曲目:https://www.uta-net.com/song/102788/
真夜中の真ん中に動物園では人知れず逢いたい相手が逢いに来る逢えない相手が逢いに来る真夜中の真ん中に動物園では人知れず逢いたい相手が逢いに来る逢えない相手が逢いに来る柵も壁も闇と同じものになった真夜中今ではもう無ない草原のはるか彼方かなたから滅ほろびた群れが連つらなってやって来る‥‥逢えない相手が逢いに来る逢えない相手が逢いに来る真夜中の羊水ようすいに動物園は浮いている逢いたい相手に逢えるまで逢えない相手に逢えるまで眠っているように外から見えるだろう真夜中はるかな国で消えた渡り鳥の迷子まいごが星に連れられて地平線を飛んで来る‥‥逢えない相手に逢えるまで逢えない相手に逢えるまで眠っているように外から見えるだろう真夜中誰だいヒトなんか呼んだのは流氷に座ってるあれはシロクマの親代おやがわりだったヒトさ‥‥逢えない相手が逢

忘れてしまえることは忘れてしまえ忘れきれないものばかり桜のもとに横たわれ抱きしめて眠らせて彼岸へ帰せ桜桜年前に桜桜見たものを話せ桜桜年前に桜桜見たものを話せ別れを告げて消えてゆくものはない思いがけないことばかり残されることが生きること抱きしめて眠らせて彼岸へ帰せ桜桜年も前に桜桜見たものを話せ桜桜年前に桜桜見たものを話せ桜桜果てしなく前に桜桜見たものを話せ

197曲目:https://www.uta-net.com/song/13308/
夢見れば人生はつらい思いが多くなるけれど夢見ずにいられないもしかしたら雲の流れは西から東つの季節をつないでゆく今日も地上に吹きつける風は左から右右から左秒毎に気が変わる予測のつかない癇癪持ちつのビルの角を曲がる度に意外な向きで吹きつけて来る私は唇かみしめて胸をそらして歩いてゆくなんでもないわ私は大丈夫どこにも隙がないなんでもないわ私は大丈夫なんでもないわどこにも隙がない夢見れば人生はつらい思いが多くなるけれど夢見ずにいられないもしかしたら弱気になった人たちは強いビル風に飛ばされる私は髪をきつく結いあげて大きなバッグを持ち直す思い出なんか何ひとつ私を助けちゃくれないわ私をいつも守ってくれるのはパウダールームの自己暗示感情的な顔にならないで誰にも弱味を知られないでなんでもないわ私は大丈夫私は傷つかないなんでもないわ私は大丈夫なんでもないわ私は傷つかない夢見れば人生はつらい思いが多くなるけれど夢見ずにいられないもしかしたらなんでもないわ私は大丈夫なんでもないわ私は傷つかない夢見れば人生はつらい思いが多くなるけれど夢見ずにいられないもしかしたら

198曲目:https://www.uta-net.com/song/54738/
子供の頃にもらったような甘い菓子など飲みこめなくて苦いグラスに溺れてるおまえを今夜もひとりひろってゆこうどのみち短い眠りなら夢かと紛う夢をみようよおいでよ受け入れる性暖める性己れのための愛を持たないおいでよ受け入れる性暖める性みかえり無用の笑みをあげよう飲みこみすぎた言葉が多過ぎて肺にあふれて心をふさぐ海の深くで黙りこむおまえを今夜もひとりひろってゆこうどのみち終わらぬ旅路なら夢とも知らぬ夢をみようよおいでよ受け入れる性暖める性己れのための愛を持たないおいでよ受け入れる性暖める性みかえり無用の笑みをあげよう

1

18曲目:https://www.uta-net.com/song/54562/
雪気がつけばいつしかなぜこんな夜に降るのいまあの人の命が永い別れ私に告げましたあの人が旅立つ前に私が投げつけたわがままはいつかつぐなうはずでした抱いたまま消えてしまうなんて雪気がつけばいつしかなぜこんな夜に降るのいまあの人の命が永い別れ私に告げました手をさしのべればいつもそこにいてくれた人が手をさしのべても消えるまるで淡すぎる雪のようですあの人が教えるとおり歩いてくはずだった私は雪で足跡が見えない立ちすくむあなたを呼びながら手をさしのべればいつもそこにいてくれた人が手をさしのべても消えるまるで淡すぎる雪のようですあの人が教えるとおり歩いてくはずだった私は雪で足跡が見えない立ちすくむあなたを呼びながら雪気がつけばいつしかなぜこんな夜に降るのいまあの人の命が永い別れ私に告げました

19曲目:https://www.uta-net.com/song/102768/
迷惑めいわくでなければ傍そばにいて車を拾うまで雪の中これきりと心で決めている私の最後のわがまま聞いてね灯あかり溢あふれる窓からは疑いもしない歌がこぼれ来る今日の主役は何処どこ誰かが気づいて探しに来るまで雪傘の柄えに指を添えて思い出を返しましょう足跡あしあと消しながら後あとずさる雪の上逃げる小ギツネみたいに小枝の代かわりに嘘を抱いて思い出消しながら遠ざかりましょう今日を祝う人が居いてくれるのなら安心できるわいつまで人ずつなんて良くないことだわ心配したのよ雪傘の柄えに指を添えてゆく時を聞いているありとあらゆる悲しいことからあなたが守ってくれていたんだね当り前のように暮らしたあの頃アリガトって伝え忘れたね今日を祝う人が居いてくれるのなら安心できるわいつまで人ずつなんて良くないことだわ心配したのよ雪傘の柄えに指を添えてゆく時を聞いている思い出全部アリガト

20曲目:https://www.uta-net.com/song/55052/
長いあいだ待ちこがれてたあたしにでもだれか残っているかとこんないい年になってしまってもおとぎばなしじみたことを信じてあきらめかけては気をとり直して待ちこがれ続けてたやっと運命の人に逢えたなんて勝手に思い込んだ夢はいつでもあたしの味方あたしに都合のいいことばかりよ夢はいつでもあたしの味方悲しいことなど何も

36曲目:https://www.uta-net.com/song/54634/
灯りの点る窓あれは盗みを退よけるため人住むふりをしてほら、じきにも点く別れの場面をはじめから描いてさかさにたどって物語を書き終えて誰にも出会わない鳥たちが眠りにつく

37曲目:https://www.uta-net.com/song/53807/
街じゅうが今夜だけのために何もかも変わろうとする夜ほんのひと月前の別れも昔のことと許される夜幸せにならなきゃならないように人は必ず創られてるとあの日あなたに聞いたのに恋人たちだけのためにすべてのドアが出迎える人ならどこへ歩こう白い息が行方に迷う街じゅうが今夜だけのために何もかも変わろうとする夜ほんのひと月前の別れも昔のことと許される夜本当の愛だけがこの夜までに結ばれるのが真実だよとあの日あなたに聞いたのに恋人たちだけのためにすべての傷は癒される今夜の願いごとは叶う愛のためにすべてが変わる街じゅうが今夜だけのために何もかも変わろうとする夜ほんのひと月前の別れも昔のことと許される夜恋人たちだけのためにすべてのドアが出迎える

38曲目:https://www.uta-net.com/song/51925/
右の翼は夜忘れさせる夜つらさ悲しさを忘れさせる翼左翼は海思い出させる海忘れたくなかった人を映す海逃げなさい心よ怖れの国から闇色翼に抱き守られながら逃げなさい心よ憂いの国から時も届かない夢へ逃げなさい歌ってもらえるあてがなければ人は自ら歌びとになるどんなにひどい雨の中でも自分の声は聞こえるからねララバイララバイ眠れ心ララバイララバイすぐ明日になるララバイララバイ眠れ心ララバイララバイすぐ明日になる思い出は翼に変われるだろうかつらさ悲しさも翼になるだろうか思い出がなければ笑えるだろうか怖れや憂いを笑えるだろうか歌ってもらえるあてがなければ人は自ら歌びとになるどんなにひどい雨の中でも自分の声は聞こえるからねララバイララバイ眠れ心ララバイララバイすぐ明日になるララバイララバイ眠れ心ララバイララバイすぐ明日になる歌ってもらえるあてがなければ人は自ら歌びとになるどんなにひどい雨の中でも自分の声は聞こえるからねララバイララバイ眠れ心ララバイララバイすぐ明日になるララバイララバイ眠れ心ララバイララバイすぐ明日になるララバイララバイ眠れ心ララバイララバイすぐ明

58曲目:https://www.uta-net.com/song/53858/
涙を見せてはいけないと教えられたのねそんなことないそんなことないそばに誰がいるのか次第男には女より泣きたいことが多いからあなたが泣くときはわたしは空を見ようあなたが泣きやめばふたりで空を見ようもう愛だとか恋だとかむずかしく言わないでわたしの子供になりなさいもう愛だとか恋だとかむずかしく言わないでわたしの子供になりなさい誰にも誉めてはもらわない石の下の石そんな日もあるそんな日もある明日は明日のために来る男は女には言わないことが多いから疲れているのならだまって抱いていようおそれているのならいつまでも抱いていようもう愛だとか恋だとかむずかしく言わないでわたしの子供になりなさいもう愛だとか恋だとかむずかしく言わないでわたしの子供になりなさいもう愛だとか恋だとかむずかしく言わないでわたしの子供になりなさいもう愛だとか恋だとかむずかしく言わないでわたしの子供になりなさい

59曲目:https://www.uta-net.com/song/54884/
愛しさ余れば憎さがつのるあれはうそっぱち愛しさ足らずたとえねぎらいひとつ返されなくても嘆くようじゃまだまだハンパな恋さだから応えられなくてごめんそんなふうに悩まないで遙か離れてしまっても泣き顔よりはいいほんとさ笑ってよエンジェル夢の中エンジェル君の笑顔消えてくれるなそれだけが願いさ笑ってよエンジェル夢の中エンジェル君の笑顔消えてくれるなそれだけが願いささよなら三角また来て四角こんど会う時はなおさら深く君を打つ冬風の風上に立ちやすらかに暮らせと祈り続けようだから思い出せなくてごめんそんなふうに悩まないで遙か離れてしまっても恋しさは育つよほんとさ笑ってよエンジェル夢の中エンジェル君の笑顔消えてくれるなそれだけが願いさ笑ってよエンジェル夢の中エンジェル君の笑顔消えてくれるなそれだけが願いささよなら三角また来て四角こんど会う時はなおさら深くさよなら三角また来て四角

60曲目:https://www.uta-net.com/song/8021/
笑わせるじゃないかあたしときたらあの人がそれとなくうるさがっているのに笑わせるじゃないかあたしときたら泣きついてじゃれついてままごと気分誰か教えてやれよと声がする気がついているわ暗闇ガラスに映ってるものみんなわかっ